In [1]:
import arcgis

In [28]:
org = "https://arcgis.com"
username = "<username>"
password = "<password>"
# Example: http://services.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer
service_to_check = "<service or layer url>"
service_without_sub_layer = "".join(service_to_check.split("//")[-1].split('/')[:-1])
# Authenticate with the org and create the GIS object
gis = arcgis.gis.GIS(org, username, password)

In [29]:
# The list to store the webmaps that use the service
webmaps_with_service = []
# Search for first 1000 webmaps
webmaps = gis.content.search('type:("Web Map" NOT "Web Mapping Application")', max_items=1000)
# Iterate over each map
for webmap in webmaps:
    service_in_map = False
    # Get the webmap json
    data = webmap.get_data()
    if "operationalLayers" in data:
        # Iterate over the the operational layers
        for operational_layer in data["operationalLayers"]:
            # Check if the url matches the service (match with sub-layer or not)
            if "url" in operational_layer and operational_layer["url"]:
                layer_with_sub_layer = operational_layer["url"].split("//")[-1]
                layer_without_sub_layer = "/".join(operational_layer["url"].split("//")[-1].split('/')[:-1])
                if layer_without_sub_layer in service_to_check or \
                        layer_without_sub_layer in service_without_sub_layer or \
                        layer_with_sub_layer in service_to_check or \
                        layer_without_sub_layer in service_without_sub_layer:
                    webmaps_with_service.append(webmap)
                    service_in_map = True
                    break
    if not service_in_map and "baseMap" in data:
        if "baseMapLayers" in data["baseMap"]:
            # Iterate over the the basemap layers
            for basemap_layer in data["baseMap"]["baseMapLayers"]:
                # Check if the url matches the service (match with sub-layer or not)
                if "url" in basemap_layer and basemap_layer["url"]:
                    layer_with_sub_layer = basemap_layer["url"].split("//")[-1]
                    layer_without_sub_layer = "/".join(basemap_layer["url"].split("//")[-1].split('/')[:-1])
                    if layer_without_sub_layer in service_to_check or \
                            layer_without_sub_layer in service_without_sub_layer or \
                            layer_with_sub_layer in service_to_check or \
                            layer_without_sub_layer in service_without_sub_layer:
                        webmaps_with_service.append(webmap)
                        service_in_map = True
                        break
# Print the webmap ids that use the service
if webmaps_with_service:
    print("Webmap ids that use this service")
    for webmap in webmaps_with_service:
        print("{}(id:{})".format(webmap.title, webmap.id))
else:
    print("No webmaps use this service")